### Sql관련 처리

In [44]:
import mysql.connector


conn = mysql.connector.connect(
    host = "ds.amazonaws.com",
    port = 3306,
    user = "",
    password = "",
    database = "project_merge"
)

In [45]:
import pandas as pd

In [46]:
cursor = conn.cursor(buffered=True)


##### 회사명에서 (주) 뺴기

In [14]:
df = pd.read_csv("project_merge.csv")


In [15]:
df['기업이름'] = df['기업이름'].str.replace('(주)', '')

In [7]:
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector:///project_merge')

df.to_sql(name='project_merge', con=engine, if_exists='replace', index=False)

944

In [16]:
cursor.execute("select * from project_merge limit 3")

result = cursor.fetchall()
for row in result:
    print(row)

('컨트롤웍스', '2년', '연봉업계평균이상', '서울 강남구 언주로81길 13 진일빌딩 3층', '자율주행', '[2년 이상] 자율주행 Front-End 개발자', '[자격요건], 학사 이상 (컴퓨터 공학/감성 및 인간공학 등 전공), 프론트엔드 개발 경력 2년 이상, 프로그래밍 능력 (HTML, CSS, JavaScript, etc.)[채용절차], 서류전형   실무·임원면접  채용검진  최종합격(상황에 따라 실무면접이 별도로 진행될 수 있습니다.)[제출서류], 이력서, 자기소개서, 최종학력 성적증명서(90일 이내 발급), 포트폴리오(선택), 이력서 내 희망연봉 필수기재[근무조건], 근무형태: 정규직, 근무일시: 주 5일(월~금), 시차출근제, 급여조건: 협의 후 결정,  IT기기, 자동차 등 제조업 기반 UX/UI 디자인 경력 보유자,  2D/3D 디자인 툴 및 프로토타입 툴 직무 경험 보유자,  웹 어플리케이션 디자인/설계 경험,  Linux 상 프로그램 개발 능력, 자율주행관련 UI/UX 컨셉 기획- 자율주행 Driving Experience에 적합한 컨셉 기획- 자율주행 UI 프로토타입 설계, 자율주행 프론트엔드 개발- 자율주행시 탑승자와 시스템간 UI 구현- 자율주행 인터페이스 유지 및 보수', 1)
('뷰런테크놀로지', '-', '-', '서초구 강남대로 311, 19층', '자율주행', '자율주행 인지 개발자', ', 전자전기/컴퓨터공학 학사 이상 또는 이와 비슷한 동종업계 경력, C/C++ 또는 Python 개발, MS or higher degree in CS/CE/EE(Candidate) or equivalent industry experience, Experience in the field of deep learning/ machine learning, Strong programming skills in C/C++ or Python, 전자전기/컴퓨터 공학관련 석사 이상학위, 라이다/비전센서 인지(분류 등) 관련 프로젝트 경험, ADAS/A

In [9]:
df.to_csv('project_merge.csv', index=False)

In [10]:
conn.commit()

##### job_planet, project_merge inner join활용 기업이름 일치하는 값들 찾기

In [29]:
query = """
SELECT j.* FROM job_planet AS j 
INNER JOIN project_merge AS p ON TRIM(j.기업이름) = TRIM(p.기업이름) 

"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result


,기업이름,분야,평점,연봉,2차산업군
0,인피닉,IT/웹/통신,2.1,2542,IT
1,슈어소프트테크,IT/웹/통신,3.1,3948,IT
2,슈어소프트테크,IT/웹/통신,3.1,3948,IT
3,슈어소프트테크,IT/웹/통신,3.1,3948,IT
4,슈어소프트테크,IT/웹/통신,3.1,3948,IT
...,...,...,...,...,...
343,에바,제조/화학,0.0,0,전기/전자/제어
344,디에스케이,제조/화학,0.0,3708,전기/전자/제어
345,니어스랩,제조/화학,3.1,4886,조선/항공/우주
346,니어스랩,제조/화학,3.1,4886,조선/항공/우주


In [37]:
#Remove duplicate rows of company name
df_result.drop_duplicates(subset=['기업이름'], keep='first', inplace=True)
len(df_result)

118

In [31]:
df_result.to_csv('pj_innerjoin.csv', index=False)

##### innerjoin과 나머지 값들의 평점 및 연봉 비교

In [32]:
import pandas as pd
df = pd.read_csv("pj_innerjoin.csv")
df.head()

,기업이름,분야,평점,연봉,2차산업군
0,인피닉,IT/웹/통신,2.1,2542,IT
1,슈어소프트테크,IT/웹/통신,3.1,3948,IT
2,크래프톤,IT/웹/통신,3.4,5179,IT
3,파수,IT/웹/통신,3.1,4298,IT
4,와디즈,IT/웹/통신,2.9,3852,IT


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기업이름    118 non-null    object 
 1   분야      118 non-null    object 
 2   평점      118 non-null    float64
 3   연봉      118 non-null    int64  
 4   2차산업군   118 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.7+ KB


In [34]:
import mysql.connector
conn = mysql.connector.connect(
    host = "",
    port = 3306,
    user = "",
    password = ",
    database = "project_merge"
)
cursor = conn.cursor(buffered=True)

In [ ]:
create_table_query = """
CREATE TABLE inner_join (
    `기업이름` VARCHAR(50),
    `분야` VARCHAR(30),
    `평점` float,
    `연봉` int,
    `2차산업군` VARCHAR(30)
);
"""
cursor.execute(create_table_query)

In [40]:
# SQL INSERT
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO inner_join (`기업이름`, `분야`, `평점`, `연봉`, `2차산업군`)
    VALUES (%s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (row['기업이름'], row['분야'], float(row['평점']), int(row['연봉']), row['2차산업군']))

In [41]:
cursor.execute("select * from inner_join limit 3;")
result = cursor.fetchall()
for row in result:
    print(row)
conn.commit()

('인피닉 ', 'IT/웹/통신', 2.1, 2542, 'IT')
('슈어소프트테크 ', 'IT/웹/통신', 3.1, 3948, 'IT')
('크래프톤 ', 'IT/웹/통신', 3.4, 5179, 'IT')


In [42]:
#inner_join table (a collection of values common to the combined data on the JobPlanet)
query = """
SELECT AVG(j.연봉) AS 평균연봉, AVG(j.평점) AS 평균평점
FROM inner_join AS j

"""

df_result = pd.read_sql(query, conn)
df_result

/tmp/ipykernel_3160/1357313640.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,평균연봉,평균평점
0,3612.3898,2.914407


In [43]:
#Average annual salary and average score by field for the remaining jab_planet values excluding #inner_jointable values
query = """
SELECT j.분야, AVG(j.연봉) AS 평균연봉, AVG(j.평점) AS 평균평점
FROM job_planet AS j
WHERE j.기업이름 NOT IN (SELECT i.기업이름 FROM inner_join AS i)
GROUP BY j.분야;
"""

df_result = pd.read_sql(query, conn)
df_result


/tmp/ipykernel_3160/1416872587.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,분야,평균연봉,평균평점
0,IT/웹/통신,2839.6097,2.586564
1,건설업,2873.0689,2.244350
2,제조/화학,3371.5677,2.327876


* cor_growth 자료랑, job_planet 자료를 조인해서, job_planet 기업이름이랑 cor_growth 기업이름이랑 매칭 --> 가능하면? 사람인이랑도 병합 이후,
* cor_growth에 추가 컬럼을 생성 :
1. ((자본총계1-자본총계3) / 자본총계3) * 100: 자본총계성장률
 2.  ((매출액1 - 매출액3) /매출액3) * 100 : 매출액성장률
3. ((순이익1 - 순이익3) /순이익3) * 100: 순이익성장률
1. orderby 순이익
 2. orderby 매출액성장률
3. orderby 순이익성장률

In [47]:
#5 outputs excluding values with cor_growth 0
import pandas as pd

query = """select * from cor_growth 
WHERE 매출액 != 0 AND 평균연봉 != 0 AND 자본총계1 != 0 AND 자본총계2 != 0 AND 자본총계3 != 0 AND 매출액1 != 0 
AND 매출액2 != 0  AND 매출액3 != 0 AND 순이익1 != 0 AND 순이익2 != 0 AND 순이익3 != 0 AND 기준년도 != 0 
limit 5;"""

df_result = pd.read_sql(query, conn)
df_result

/tmp/ipykernel_3051/4090599442.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,기업이름,기업규모,매출액,평균연봉,자본총계1,자본총계2,자본총계3,매출액1,매출액2,매출액3,순이익1,순이익2,순이익3,기준년도,키워드,인덱스
0,코가로보틱스,중소기업,37179,4578,2398977,101575,77147,37179,156842,74946,-835304,60623,37414,2022,자율주행,5
1,뷰런테크놀로지,중소기업,45919,4763,773070,939785,89852,34091,17350,18852,-266673,-44582,-10186,2022,자율주행,8
2,체이시로보틱스,중소기업,55922,2676,24567,21618,21484,55922,30082,32682,2948,134,-215,2022,자율주행,10
3,포티투닷 주식회사,중소기업,330478,7291,3650495,9432812,2462012,330478,40203,2506,-6118762,-3452141,-2147287,2022,자율주행,11
4,피너클써치,중소기업,120793,3384,99679,97840,71001,120793,109519,110621,1839,26838,19394,2018,자율주행,15


In [7]:
# Finding the Net Income Growth Index after job_planet, cor_growth join
query = """
SELECT c.기업이름, c.키워드, j.2차산업군, j.연봉, j.평점, ((c.순이익1 - c.순이익3)/c.순이익3) as 순이익성장지수
FROM cor_growth AS c
INNER JOIN job_planet AS j ON TRIM(c.기업이름) LIKE CONCAT('%', TRIM(j.기업이름), '%')
GROUP BY c.기업이름
order by 순이익성장지수 desc, j.연봉 desc, j.평점 desc limit 10;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점,순이익성장지수
0,크래프톤,딥러닝,IT,5179,3.4,93.5633
1,포인드,딥러닝,IT,0,4.0,26.4759
2,뷰런테크놀로지,딥러닝,IT,0,2.7,25.1803
3,프로메카,로봇제어,기계/설비/자동차,6312,2.5,22.3302
4,메디픽셀,로봇제어,기계/설비/자동차,0,3.7,10.9262
5,주식회사 에이치엘클레무브,자율주행,IT,0,0.0,9.3381
6,주식회사리얼게인,로봇제어,전기/전자/제어,3827,2.5,6.1678
7,아이엠티에프에이,로봇제어,기계/설비/자동차,3229,2.0,3.8172
8,주식회사리하이,드론,IT,0,3.9,2.9469
9,에이치알맨파워그룹,로봇제어,IT,2779,3.6,2.7821


In [6]:
# Finding Sales Growth Index After Job_planet, Cor_growth join
query = """
SELECT c.기업이름, c.키워드, j.2차산업군, j.연봉, j.평점, ((c.매출액1 - c.매출액3)/c.매출액3) as 매출액성장지수
FROM cor_growth AS c
INNER JOIN job_planet AS j ON TRIM(c.기업이름) LIKE CONCAT('%', TRIM(j.기업이름), '%')
GROUP BY c.기업이름
order by 매출액성장지수 desc, j.연봉 desc, j.평점 desc limit 10;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점,매출액성장지수
0,포티투닷 주식회사,딥러닝,IT,6550,2.5,130.8747
1,에바,로봇제어,전기/전자/제어,0,0.0,42.9502
2,고스트패스,딥러닝,IT,3563,1.0,30.2486
3,와디즈,딥러닝,IT,3852,2.9,27.6172
4,크래프톤,딥러닝,IT,5179,3.4,17.5230
5,쉬모스랩,딥러닝,건설업,0,0.0,14.7036
6,루덴스파트너스,로봇제어,IT,0,3.0,14.4058
7,반프,자율주행,전기/전자/제어,0,4.6,12.7333
8,오픈엣지테크놀로지,자율주행,IT,5782,2.0,8.1965
9,무스마,딥러닝,IT,3351,3.8,5.0306


In [8]:
# Calculating the total capital growth index after job_planet, cor_growth join
query = """
SELECT c.기업이름, c.키워드, j.2차산업군, j.연봉, j.평점,((c.자본총계1 - c.자본총계3)/c.자본총계3) as 자본총계성장지수
FROM cor_growth AS c
INNER JOIN job_planet AS j ON TRIM(c.기업이름) LIKE CONCAT('%', TRIM(j.기업이름), '%')
GROUP BY c.기업이름
order by 자본총계성장지수 desc, j.연봉 desc, j.평점 desc limit 10;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점,자본총계성장지수
0,반프,자율주행,전기/전자/제어,0,4.6,41.1214
1,프로메카,로봇제어,기계/설비/자동차,6312,2.5,40.8522
2,코가로보틱스,로봇제어,IT,3365,1.8,30.0962
3,메디픽셀,로봇제어,기계/설비/자동차,0,3.7,14.4120
4,주식회사 딥브레인에이아이,딥러닝,IT,4226,3.2,11.1238
5,코난테크놀로지,딥러닝,IT,3863,2.6,8.2310
6,뷰런테크놀로지,딥러닝,IT,0,2.7,7.6038
7,주식회사리하이,드론,IT,0,3.9,7.2469
8,주식회사 한컴어썸텍,로봇제어,IT,4203,2.5,4.5662
9,포인드,딥러닝,IT,0,4.0,3.7364


In [23]:
# Obtain total growth index after job_planet, cor_growth join
query = """
SELECT c.기업이름, c.키워드, j.2차산업군, j.연봉, j.평점,
  (((c.순이익1 - c.순이익3) / c.순이익3) + ((c.매출액1 - c.매출액3) / c.매출액3) + ((c.자본총계1 - c.자본총계3) / c.자본총계3)) as 총성장지수
FROM cor_growth AS c
INNER JOIN job_planet AS j ON TRIM(c.기업이름) LIKE CONCAT('%', TRIM(j.기업이름), '%')
GROUP BY c.기업이름
ORDER BY 총성장지수 DESC, j.연봉 DESC, j.평점 DESC
LIMIT 10;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점,총성장지수
0,포티투닷 주식회사,딥러닝,IT,6550,2.5,133.2070
1,크래프톤,딥러닝,IT,5179,3.4,114.2853
2,프로메카,로봇제어,기계/설비/자동차,6312,2.5,66.6813
3,에바,로봇제어,전기/전자/제어,0,0.0,47.1051
4,뷰런테크놀로지,딥러닝,IT,0,2.7,33.5925
5,포인드,딥러닝,IT,0,4.0,31.0947
6,고스트패스,딥러닝,IT,3563,1.0,29.4795
7,와디즈,딥러닝,IT,3852,2.9,26.1464
8,메디픽셀,로봇제어,기계/설비/자동차,0,3.7,24.9265
9,쉬모스랩,딥러닝,건설업,0,0.0,14.2154


In [20]:
# Obtaining the total growth index after job_planet, cor_growth join (company size: medium-sized companies only)
query = """
SELECT c.기업이름, c.키워드, j.2차산업군, j.연봉, j.평점,c.기업규모,
(((c.순이익1 - c.순이익3) / c.순이익3) + ((c.매출액1 - c.매출액3) / c.매출액3) + ((c.자본총계1 - c.자본총계3) / c.자본총계3)) as 총성장지수
FROM cor_growth AS c
INNER JOIN job_planet AS j ON TRIM(c.기업이름) LIKE CONCAT('%', TRIM(j.기업이름), '%')
where c.기업규모 = '중견기업'
GROUP BY c.기업이름, c.기업규모
order by 총성장지수 desc, j.연봉 desc, j.평점 desc limit 10;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점,기업규모,총성장지수
0,크래프톤,딥러닝,IT,5179,3.4,중견기업,114.2853
1,주식회사 에이치엘클레무브,자율주행,IT,0,0.0,중견기업,11.0755
2,유진로봇,로봇제어,전기/전자/제어,4407,2.5,중견기업,-0.3941
3,아이엠아이크리티컬엔지어링코리아,로봇제어,IT,3499,3.2,중견기업,-0.4861
4,AJ네트웍스 주식회사,로봇제어,IT,0,2.0,중견기업,-0.5601
5,안진회계법인,자율주행,전기/전자/제어,4742,3.0,중견기업,-1.0907
6,아이씨디,로봇제어,반도체/디스플레이,4040,2.4,중견기업,-1.9998
7,유베이스,로봇제어,반도체/디스플레이,3426,2.2,중견기업,-3.3234
8,텔레칩스,자율주행,전기/전자/제어,5108,3.3,중견기업,-4.5476
9,주식회사 디알비동일,로봇제어,건설업,4183,1.0,중견기업,-4.7488


* project_merge + job_planet join 후 pj_join 만들기 
* job_planet - pj_join > pj_difference.csv

In [10]:
# project_merge, job_planet join
query = """
SELECT p.기업이름, p.키워드, j.2차산업군, j.연봉, j.평점
FROM project_merge AS p
INNER JOIN job_planet AS j ON TRIM(j.기업이름) LIKE CONCAT('%', TRIM(p.기업이름), '%')
GROUP BY p.기업이름;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,키워드,2차산업군,연봉,평점
0,인피닉,자율주행,IT,2542,2.1
1,슈어소프트테크,딥러닝,IT,3948,3.1
2,크래프톤,딥러닝,IT,5179,3.4
3,파수,딥러닝,IT,4298,3.1
4,와디즈,딥러닝,IT,3852,2.9
...,...,...,...,...,...
123,엠투아이코퍼레이션,로봇제어,전기/전자/제어,3948,3.6
124,캔탑스,딥러닝,전기/전자/제어,3755,2.7
125,오토엘,로봇제어,전기/전자/제어,5452,2.8
126,반프,자율주행,전기/전자/제어,0,4.6


In [ ]:
df_result.to_csv('pj_join.csv', index=False)

In [35]:
# project_merge, job_planet join(Lowest salary row extracted)
query = """
SELECT j.*
FROM project_merge AS p
INNER JOIN job_planet AS j ON TRIM(j.기업이름) LIKE CONCAT('%', TRIM(p.기업이름), '%')
WHERE j.연봉 > 0
GROUP BY p.기업이름
ORDER BY j.연봉 asc limit 1;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,분야,평점,연봉,2차산업군
0,에이비씨랩스,IT/웹/통신,2.8,2384,IT


In [43]:
# job_planet-(project_merge, job_planet)(Lowest salary row extracted)
query = """
SELECT j.*
FROM job_planet AS j
WHERE TRIM(j.기업이름) NOT IN (
    SELECT p.기업이름
    FROM project_merge AS p
    INNER JOIN job_planet AS j2 ON TRIM(p.기업이름) LIKE CONCAT('%', TRIM(j2.기업이름), '%')
    GROUP BY p.기업이름
)
AND j.연봉 > 0
ORDER BY j.연봉 ASC
LIMIT 1;
"""

cursor.execute(query)

result = cursor.fetchall()

columns = [i[0] for i in cursor.description]
df_result = pd.DataFrame(result, columns=columns)

df_result

,기업이름,분야,평점,연봉,2차산업군
0,세명,제조/화학,3.0,1048,전기/전자/제어


In [26]:
df_result.to_csv('pj_difference.csv', index=False)

In [34]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기업이름    44 non-null     object 
 1   기업규모    44 non-null     object 
 2   매출액     44 non-null     int64  
 3   평균연봉    44 non-null     int64  
 4   자본총계1   44 non-null     int64  
 5   자본총계2   44 non-null     int64  
 6   자본총계3   44 non-null     int64  
 7   매출액1    44 non-null     int64  
 8   매출액2    44 non-null     int64  
 9   매출액3    44 non-null     int64  
 10  순이익1    44 non-null     int64  
 11  순이익2    44 non-null     int64  
 12  순이익3    44 non-null     int64  
 13  기준년도    44 non-null     int64  
 14  키워드     44 non-null     object 
 15  인덱스     44 non-null     int64  
 16  평점      44 non-null     float64
 17  연봉      44 non-null     int64  
dtypes: float64(1), int64(14), object(3)
memory usage: 6.3+ KB
